In [ ]:
import utils as my
import pandas as pd
import pubchempy as pbp
from rdkit import Chem

In [7]:
data = pd.read_csv('../data.csv')
smiles = data['smiles']
smiles_nonan = []
for i in smiles:
    if not pd.isna(i):
        smiles_nonan.append(i)

In [8]:
fingerprints = my.get_compounds(smiles_nonan, result='fingerprint', identifier='smiles')

In [23]:
print(fingerprints[0])
fingerprint_bin = bin(int(fingerprints[0], 16))[2:].zfill(881)

00000371C073318000000000000000000000000000016000000030000000000000000001C000001F00140000000C0CC19A143EC492D00000A902B5775200820000252200289821246CDA08243EC8D1918421C862B408C8C9C71080000E08000000000000001000000000000000000000000000


In [20]:
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors

# 读取分子的SMILES字符串
smiles = smiles_nonan[0]

# 从SMILES字符串创建分子对象
molecule = Chem.MolFromSmiles(smiles)

# 生成PubChem分子指纹
pubchem_fingerprint = rdMolDescriptors.GetMorganFingerprintAsBitVect(molecule, 2, nBits=881)

# 将指纹转换为列表
pubchem_fingerprint_list = list(pubchem_fingerprint)

# 打印指纹
f2 = pubchem_fingerprint_list

In [22]:
fingerprint_bin, str(f2)

('1101110001110000000111001100110001100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001011000000000000000000000000000000011000000000000000000000000000000000000000000000000000000000000000000000000000111000000000000000000000000011111000000000001010000000000000000000000000000001100000011001100000110011010000101000011111011000100100100101101000000000000000000001010100100000010101101010111011101010010000000001000001000000000000000000010010100100010000000000010100010011000001000010010010001101100110110100000100000100100001111101100100011010001100100011000010000100001110010000110001010110100000010001100100011001001110001110001000010000000000000000000111000001000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
 '[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1